In [59]:
from math import radians

import numpy as np
import plotly.graph_objects as go
import pandas as pd
import pvlib
from pvlib import solarposition


In [60]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [61]:
year_range = pd.date_range("01/01/2020 00:00:00", "31/12/2020 00:00:00", freq="H", tz="Europe/Oslo")
lat, lon = 59.946247,10.761360
data = solarposition.get_solarposition(year_range,lat, lon)

In [62]:
data.head()

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
2020-01-01 00:00:00+01:00,142.949853,142.949853,-52.949853,-52.949853,352.344323,-3.061851
2020-01-01 01:00:00+01:00,142.452243,142.452243,-52.452243,-52.452243,15.183254,-3.081724
2020-01-01 02:00:00+01:00,139.172168,139.172168,-49.172168,-49.172168,36.472129,-3.101587
2020-01-01 03:00:00+01:00,133.793610,133.793610,-43.793610,-43.793610,54.986541,-3.121441
2020-01-01 04:00:00+01:00,127.113558,127.113558,-37.113558,-37.113558,70.893202,-3.141287


In [63]:
def create_year_data(base_year=2020):
    year_range = pd.date_range(f"01/01/{base_year} 00:00:00", f"31/12/{base_year} 00:00:00", freq="H", tz="Europe/Oslo")
    lat, lon = 59.946247,10.761360
    data = solarposition.get_solarposition(year_range,lat, lon)
    return data

In [64]:
def create_today_data(date="2020.02.05", lat=59.946247, lon=10.761360, c=0.001):
    data = create_year_data(date.split(".")[0])
    sun_hours = data[data["elevation"] > 0]
    today_data = sun_hours.loc[date]
    lon_factor = 1/np.cos(np.radians(lat))
    today_data["lat"] = lat + c*np.cos(np.radians(today_data["azimuth"]))*np.cos(np.radians(today_data["elevation"]))
    today_data["lon"] = lon + c*lon_factor*np.sin(np.radians(today_data["azimuth"]))*np.cos(np.radians(today_data["elevation"]))

    return today_data
    

In [66]:
def create_scattermap(today_data):
    hours = [str(hour) for hour in today_data.index.hour]

    return go.Scattermapbox(
        lat=today_data["lat"],
        lon=today_data["lon"],
        mode="markers+text",
        marker=go.scattermapbox.Marker(size=22, color="orange"),
        text=hours,
        name="Hour of the day",
        textfont=dict(family="sans serif", size=18, color="black"),
    )

In [67]:
def create_animated_map(lat=59.946247, lon=10.761360):
    mapbox_access_token = open("mapbox_token.txt").read()

    fig = go.Figure(data=go.Scattermapbox(
        lat=[lat],
        lon=[lon],
        mode='markers+text',
        name="Position",
        marker=go.scattermapbox.Marker(
            size=14,
            color="blue"
        )
    ),
        layout=go.Layout(autosize=True,
                         hovermode='closest',
                         height=700,
                         mapbox=go.layout.Mapbox(
                             accesstoken=mapbox_access_token,
                             bearing=0,
                             center=go.layout.mapbox.Center(
                                 lat=lat,
                                 lon=lon
                             ),
                             pitch=0,
                             zoom=16
                         ),
                         updatemenus=[dict(type="buttons",
                                           buttons=[dict(label="Play",
                                                         method="animate",
                                                         args=[None])])]),
        frames=[go.Frame(
            data=[create_scattermap(create_today_data(date)) for date in ["2020.02.05",
                                                                          "2020.03.05",
                                                                          "2020.04.05",
                                                                          "2020.05.05",
                                                                          "2020.06.05"]])
        ],
    )
    fig.show()


In [97]:
def create_map(date="2020.02.05", lat=59.946247, lon=10.761360):
    today_data = create_today_data(date=date)
    hours = [str(hour) for hour in today_data.index.hour]
    mapbox_access_token = open("mapbox_token.txt").read()

    fig = go.Figure(go.Scattermapbox(
            lat=today_data["lat"],
            lon=today_data["lon"],
            mode='markers+text',
            marker=go.scattermapbox.Marker(
                size=22,
                color="orange"
            ),
            text=hours,
            name = "Hour of the day",
            textfont=dict(
            family="sans serif",
            size=18,
            color="black"
            )
        ))

    fig.add_trace(go.Scattermapbox(
            lat=[lat],
            lon=[lon],
            mode='markers+text',
            name = "Position",
            marker=go.scattermapbox.Marker(
                size=14,
                color="blue"
            )
        ))
    styles = ["basic", "streets", "outdoors", "light", "dark", "satellite", "satellite-streets"]
    fig.update_layout(
        autosize=True,
        hovermode='closest',
        height=700,
        mapbox=go.layout.Mapbox(
            accesstoken=mapbox_access_token,
            style=styles[-5],
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=lat,
                lon=lon
            ),
            pitch=0,
            zoom=16
        ),

    )

    fig.show()

In [98]:
create_map(date="2020.02.27")

In [70]:
mapbox_access_token = open("mapbox_token.txt").read()

fig = go.Figure(go.Scattermapbox(
        lat=today_data["lat"],
        lon=today_data["lon"],
        mode='markers+text',
        marker=go.scattermapbox.Marker(
            size=22,
            color="orange"
        ),
        text=hours,
        textfont=dict(
        family="sans serif",
        size=18,
        color="black"
    )
    ))

fig.add_trace(go.Scattermapbox(
        lat=[lat],
        lon=[lon],
        mode='markers+text',
        marker=go.scattermapbox.Marker(
            size=22,
            color="black"
        )
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    height=700,
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=lat,
            lon=lon
        ),
        pitch=0,
        zoom=17
    ),
)

fig.show()

NameError: name 'today_data' is not defined